```
This notebook sets up and runs a set of benchmarks to compare
different numerical discretizations of the SWEs

Copyright (C) 2016  SINTEF ICT

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <http://www.gnu.org/licenses/>.
```

## Topographic Rossby adjustment

Using constant Coriolis but linear bottom topography.

The results should be similar to the case with linear coriolis parameter.

#### Import modules and set up environment

In [ ]:
#Lets have matplotlib "inline"
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

#Import packages we need
import numpy as np
from matplotlib import animation, rc
from matplotlib import pyplot as plt

import os
import pyopencl
import datetime
import sys

#Finally, import our simulator
from SWESimulators import FBL, CTCS, DataOutput

#Set large figure sizes
rc('figure', figsize=(16.0, 12.0))
rc('animation', html='html5')

#Import our simulator
from SWESimulators import FBL, CTCS,KP07, CDKLM16, DataOutput, PlotHelper, Common
#Import initial condition and bathymetry generating functions:
from SWESimulators.BathymetryAndICs import *

In [ ]:
#Make sure we get compiler output from OpenCL
os.environ["PYOPENCL_COMPILER_OUTPUT"] = "1"

#Set which CL device to use, and disable kernel caching
if (str.lower(sys.platform).startswith("linux")):
    os.environ["PYOPENCL_CTX"] = "0"
else:
    os.environ["PYOPENCL_CTX"] = "1"
os.environ["CUDA_CACHE_DISABLE"] = "1"
os.environ["PYOPENCL_COMPILER_OUTPUT"] = "1"
os.environ["PYOPENCL_NO_CACHE"] = "1"

#Create OpenCL context
cl_ctx = pyopencl.create_some_context()
print "Using ", cl_ctx.devices[0].name

In [ ]:
#Create output directory for images
imgdir='images_linearCoriolis_' + datetime.datetime.now().strftime("%Y_%m_%d-%H_%M_%S")
os.makedirs(imgdir)
print "Saving images to " + imgdir

## Parameters

Kai and Göran: Go nuts and make changes here :)

In [ ]:
nx = 400
ny = 350

dx = 20000.0
dy = 20000.0

# The simulation have the following structure:
# for 1:T
#     simulate until t = t0 + sub_t 
#     add plot to animation

#T = 200
#sub_t = 300.0

dt = 100

#T = 100
#sub_t = dt*300
#writeNetCDF=False

T = 150
sub_t = dt*400
make_netCDF=True

#T = 20
#sub_t = dt*400
#make_netCDF = False

g = 9.81
r = 0.0

# Eddy viscocity parameter
A = 0

# Coriolis parameters: f + beta * y
f = 8.0e-5
beta = 0.0
not_to_be_used_beta = 2.0e-11
alpha = not_to_be_used_beta / f
y0_reference_cell = ny/2

wind = Common.WindStressParams(type=99) # type = 99 corresponds to no wind.

# Boundary condions
# 1: reflective wall, 
# 2: periodic, 
# 3: open (flow relaxation scheme)
# 4: periodic east-west, open in north-south
# 5: periodic north-south, open in east-west
bcSettings = 3


waterHeight = 50

#Calculate radius from center for plotting
x_center = dx*nx*0.5
y_center = dy*ny*0.5
y_coords, x_coords = np.mgrid[0:ny*dy:dy, 0:nx*dx:dx]
x_coords = np.subtract(x_coords, x_center)
y_coords = np.subtract(y_coords, y_center)
radius = np.sqrt(np.multiply(x_coords, x_coords) + np.multiply(y_coords, y_coords))

bump_posx = 0.8
bump_posy = 0.5
bump_width_factor = 2500
bump_height = 0.25

reload(Common)

In [ ]:
def initializeBalancedVelocityFieldStaggered(eta, H, hu, hv, f, beta, g, nx, ny, dx, dy, ghosts):
    for j in range(1, ny+ghosts[0]+ghosts[2]-1):
        coriolis = f + beta*j*dy
        for i in range(1, nx+ghosts[1]+ghosts[3]-1):
            eta_pluss = (eta[j+1, i] + eta[j+1, i-1])/2.0
            eta_minus = (eta[j-1, i] + eta[j-1, i-1])/2.0
            h_mid = (eta[j,i] + H[j,i] + eta[j, i-1] + H[j,i-1])/2.0
            hu[j,i] = -(g/coriolis)*h_mid*(eta_pluss - eta_minus)/(2.0*dy)
    for j in range(1, ny+ghosts[0]+ghosts[2]-1):
        coriolis = f + beta*j*dy
        for i in range(1, nx+ghosts[1]+ghosts[3]-1):
            eta_pluss = (eta[j, i+1] + eta[j-1, i+1])/2.0
            eta_minus = (eta[j, i-1] + eta[j-1, i-1])/2.0
            h_mid = (eta[j,i] + H[j,i] + eta[j-1,i] + H[j-1,i])/2.0
            hv[j,i] =  (g/coriolis)*h_mid*(eta_pluss - eta_minus)/(2.0*dx)
            
def initializeBalancedVelocityField(eta, H, hu, hv, f, beta, g, nx, ny, dx, dy, ghosts):
    for j in range(1, ny+ghosts[0]+ghosts[2]-1):
        coriolis = f + beta*j*dy
        for i in range(1, nx+ghosts[1]+ghosts[3]-1):
            h_mid = eta[j,i] + H[j,i]
            
            eta_diff_y = (eta[j+1, i] - eta[j-1, i])/(2.0*dy)
            #eta_diff_y = (eta[j, i] - eta[j-1,i])/(dy)
            hu[j,i] = -(g/coriolis)*h_mid*eta_diff_y
            
            eta_diff_x = (eta[j, i+1] - eta[j, i-1])/(2.0*dx)
            #eta_diff_x = (eta[j, i] - eta[j,i-1])/(dx)
            hv[j,i] = (g/coriolis)*h_mid*eta_diff_x            
            
            
# Depth:
def exponentialBathymetryYDebug(B,dx, dy, halo, alpha, high, y0, addTerm):
    ny, nx = B.shape
    for j in range(0, ny):
        y = (j - ny/2.0)*dy
        #if (y < 0):
        #    y = 0
        for i in range(0, nx):
            B[j, i] = addTerm + high*np.exp(alpha*(y0-y))
            
def exponentialBathymetryYGoran(B,dx, dy, halo, alpha, H0, y0, H1):
    ny, nx = B.shape
    for j in range(0, ny):
        y = (j - ny/2.0)*dy
        #if (y < 0):
        #    y = 0
        for i in range(0, nx):
            B[j, i] = H0*np.exp(alpha*y)
            
            
            
def exponentialBathymetryXDebug(B, dx, dy, halo, alpha, high):
    ny, nx = B.shape
    for j in range(0, ny):
        for i in range(0, nx):
            x = (j - nx/2.0)*dx
            if (x < 0):
                x = 0
            B[j, i] = high*np.exp(alpha*x)

# linear bottom in y:   H(y) = H_0 (1 - alpha(y - y0))
def linearBathymetryY(H, dx, dy, nx, ny, halo, alpha, H0, y0_reference_cell ):
    y0 = y0_reference_cell*dy
    for j in range(-halo[2], ny+halo[0]):
        y = (j + 0.5)*dy
        for i in range(-halo[3], nx+halo[1]):
            depth = H0*(1 - alpha*(y-y0))
            if depth < 3:
                depth = 3
            H[halo[2]+j, halo[3]+i] = depth   

## Centered in time, centered in space

In [ ]:
#Centered in time, centered in space

#Clean up old simulator if any:
if 'ctcs_sim' in globals():
    ctcs_sim.cleanUp()

ghosts = [1,1,1,1] # north, east, south, west
validDomain = [1,1,1,1] 
if (bcSettings == 1):
    boundaryConditions = Common.BoundaryConditions()
    # Wall boundary conditions
elif (bcSettings == 2):
    # periodic boundary conditions
    boundaryConditions = Common.BoundaryConditions(2,2,2,2)
elif bcSettings == 3:
    validDomain = [10, 10, 10, 10]
    boundaryConditions = Common.BoundaryConditions(3,3,3,3, spongeCells=validDomain)
    ghosts = [10, 10, 10, 10]
elif bcSettings == 4:
    validDomain = [10, 1, 10, 1]
    boundaryConditions = Common.BoundaryConditions(3,2,3,2, spongeCells=validDomain)
    ghosts = [10, 1, 10, 1]
elif bcSettings == 5:
    validDomain = [1, 10, 1, 10]
    boundaryConditions = Common.BoundaryConditions(2,3,2,3, spongeCells=validDomain)
    ghosts = [1, 10, 1, 10]
    
dataShape = (ny + ghosts[0]+ghosts[2], 
             nx + ghosts[1]+ghosts[3])

h0 = np.zeros(dataShape, dtype=np.float32, order='C')
eta0 = np.zeros(dataShape, dtype=np.float32, order='C');
u0 = np.zeros((dataShape[0], dataShape[1]+1), dtype=np.float32, order='C');
v0 = np.zeros((dataShape[0]+1, dataShape[1]), dtype=np.float32, order='C');    

linearBathymetryY(h0, dx, dy, nx, ny, ghosts, alpha, waterHeight, y0_reference_cell )

initializeBalancedBumpOverPoint(eta0, u0, v0, # allocated buffers to be filled with data (output)
                                nx, ny, dx, dy, ghosts, # grid data
                                bump_posx, bump_posy, # relative placement of bump center
                                bump_height, bump_width_factor, # bump information
                                f, waterHeight, # parameters defined at the bump centre (coriolis force, water depth)
                                g)



fig = plt.figure(figsize=(4,4))
plt.imshow(h0, origin="lower left")
plt.colorbar()

fig = plt.figure(figsize=(4,4))
plt.plot(h0[:,3])

print "mean h: ", np.mean(h0)
print "min h:  ", np.min(h0)
print "max h:  ", np.max(h0)

fig = plt.figure(figsize=(4,4))
plt.imshow(eta0, origin="lower left")
plt.colorbar()

fig = plt.figure(figsize=(4,4))
plt.imshow(u0, origin="lower left")
plt.colorbar()

fig = plt.figure(figsize=(4,4))
plt.imshow(v0, origin="lower left")
plt.colorbar()

#Initialize simulator
reload(CTCS)
ctcs_sim = CTCS.CTCS(cl_ctx, \
                     h0, eta0, u0, v0, \
                     nx, ny, dx, dy, dt, \
                     g, f, r, A, \
                     coriolis_beta=beta, \
                     wind_stress=wind, \
                     write_netcdf=make_netCDF, \
                     boundary_conditions=boundaryConditions )



fig = plt.figure()
plotter = PlotHelper.PlotHelper(fig, x_coords, y_coords, radius, 
                                eta0[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]] , 
                                u0[validDomain[2]:-validDomain[0], validDomain[3]+1:-validDomain[1]], 
                                v0[validDomain[2]+1:-validDomain[0], validDomain[3]:-validDomain[1]])

def animate(i):
    if (i>0):
        t = ctcs_sim.step(sub_t)
    else:
        t = 0.0
    eta1, u1, v1 = ctcs_sim.download()
    
    # To increase the amplitude of the plot, increase this brighten parameter
    brighten = 1.0
    
    plotter.plot(brighten*(eta1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]]), 
                 brighten*u1[validDomain[2]:-validDomain[0], validDomain[3]+1:-validDomain[1]], 
                 brighten*v1[validDomain[2]+1:-validDomain[0], validDomain[3]:-validDomain[1]])
    fig.suptitle("CTCS Time = " + "{:04.0f}".format(t) + " s", fontsize=18)

    if (i%20 == 0):
        print "{:03.0f}".format(100.0*i / T) + " % => t=" + str(t) +  "\tMax h: " + str(np.max(eta1)) + \
        "\tMax u: " + str(np.max(u1)) + "\tMax v: " + str(np.max(v1))
        fig.savefig(imgdir + "/{:010.0f}_ctcs.png".format(t))

anim = animation.FuncAnimation(fig, animate, range(T), interval=100)
plt.close(anim._fig)
anim

In [ ]:
if make_netCDF:
    ctcs_sim.cleanUp()

## CDKLM

In [ ]:
#Coriolis well balanced reconstruction scheme
if 'cdklm_sim' in globals():
    cdklm_sim.cleanUp()

ghosts = np.array([2,2,2,2]) # north, east, south, west
validDomain = np.array([2,2,2,2])
if (bcSettings == 1):
    boundaryConditions = Common.BoundaryConditions()
elif (bcSettings == 2):
    boundaryConditions = Common.BoundaryConditions(2,2,2,2)
elif bcSettings == 3:
    validDomain = [10, 10, 10, 10]
    boundaryConditions = Common.BoundaryConditions(3,3,3,3, spongeCells=validDomain)
    ghosts = [10, 10, 10, 10]
        
dataShape = (ny + ghosts[0]+ghosts[2], 
             nx + ghosts[1]+ghosts[3])

eta0 = np.zeros(dataShape, dtype=np.float32, order='C')
u0   = np.zeros(dataShape, dtype=np.float32, order='C');
v0   = np.zeros(dataShape, dtype=np.float32, order='C');

# Bathymetry:
Hi = np.zeros((dataShape[0]+1, dataShape[1]+1), dtype=np.float32, order='C')
linearBathymetryY(Hi, dx, dy, nx+1, ny+1, ghosts, alpha, waterHeight, y0_reference_cell+0.5 )

initializeBalancedBumpOverPoint(eta0, u0, v0, # allocated buffers to be filled with data (output)
                                nx, ny, dx, dy, ghosts, # grid data
                                bump_posx, bump_posy, # relative placement of bump center
                                bump_height, bump_width_factor, # bump information
                                f, waterHeight, # parameters defined at the bump centre (coriolis force, water depth)
                                g)

#Initialize simulator
reload(CDKLM16)
cdklm_sim = CDKLM16.CDKLM16(cl_ctx, \
                            eta0, u0, v0, Hi, \
                            nx, ny, dx, dy, dt, \
                            g, f, r, \
                            rk_order=3, \
                            coriolis_beta=beta, \
                            wind_stress=wind, \
                            boundary_conditions=boundaryConditions, \
                            write_netcdf=make_netCDF, \
                            double_precision=True, \
                            reportGeostrophicEquilibrium=False, \
                            )


fig = plt.figure()
plotter = PlotHelper.PlotHelper(fig, x_coords, y_coords, radius, 
                                eta0[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]], 
                                u0[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]], 
                                v0[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]])

def animate(i):
    if (i>0):
        t = cdklm_sim.step(sub_t)
    else:
        t = 0.0
    eta1, u1, v1 = cdklm_sim.download()
    
    # To increase the amplitude of the plot, increase this brighten parameter
    brighten = 1
    
    plotter.plot(brighten*(eta1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]]), 
                 brighten*   u1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]], 
                 brighten*   v1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]]);
    fig.suptitle("CDKLM16 Time = " + "{:04.0f}".format(t) + " s", fontsize=18)

    if (i%20 == 0):
        print "{:03.0f}".format(100*i / T) + " % => t=" + str(t) + "\tMax h: " + str(np.max(eta1)) + \
        "\tMax u: " + str(np.max(u1)) + \
        "\tMax v: " + str(np.max(v1))
        fig.savefig(imgdir + "/{:010.0f}_cdklm16.png".format(t))
             
anim = animation.FuncAnimation(fig, animate, range(T), interval=100)
plt.close(anim._fig)
anim

In [ ]:
if make_netCDF:
     cdklm_sim.cleanUp()

In [ ]:
## FBL

#Clean up old simulator if any:
if 'fbl_sim' in globals():
    fbl_sim.cleanUp()


if (bcSettings == 1):
    boundaryConditions = Common.BoundaryConditions()
    ghosts = [0,0,0,0] # north, east, south, west
    validDomain = [None, None, 0, 0]
elif (bcSettings == 2):
    boundaryConditions = Common.BoundaryConditions(2,2,2,2)
    ghosts = [1,1,0,0] # Both periodic
    validDomain = [1, 1, 0, 0]
elif bcSettings == 3:
    validDomain = [10, 10, 10, 10]
    boundaryConditions = Common.BoundaryConditions(3,3,3,3, spongeCells=validDomain)
    ghosts = [10, 10, 10, 10]
    
dataShape = (ny + ghosts[0]+ghosts[2], 
             nx + ghosts[1]+ghosts[3])

h0 = np.ones(dataShape, dtype=np.float32, order='C') * waterHeight;

eta0 = np.zeros(dataShape, dtype=np.float32);
u0 = np.zeros((dataShape[0], dataShape[1]+1), dtype=np.float32);
v0 = np.zeros((dataShape[0]+1, dataShape[1]), dtype=np.float32);

# Bathymetry:
h0 = np.zeros(dataShape, dtype=np.float32, order='C')
linearBathymetryY(h0, dx, dy, nx, ny, ghosts, alpha, waterHeight, y0_reference_cell )

initializeBalancedBumpOverPoint(eta0, u0, v0, # allocated buffers to be filled with data (output)
                                nx, ny, dx, dy, ghosts, # grid data
                                bump_posx, bump_posy, # relative placement of bump center
                                bump_height, bump_width_factor, # bump information
                                f, waterHeight, # parameters defined at the bump centre (coriolis force, water depth)
                                g)


#Initialize simulator
reload(FBL)
fbl_sim = FBL.FBL(cl_ctx, \
              h0, eta0, u0, v0, \
              nx, ny, \
              dx, dy, dt, \
              g, f, r, \
              coriolis_beta=beta,\
              wind_stress=wind, \
              boundary_conditions=boundaryConditions, \
              write_netcdf=make_netCDF)
   
fig = plt.figure()
plotter = PlotHelper.PlotHelper(fig, x_coords, y_coords, radius, \
                                eta0[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]],\
                                u0[validDomain[2]:-validDomain[0], validDomain[3]-1:-validDomain[1]+1], \
                                v0[validDomain[2]-1:-validDomain[0]+1, validDomain[3]:-validDomain[1]])
def animate(i):
    if (i>0):
        t = fbl_sim.step(sub_t)
    else:
        t = 0.0
    eta1, u1, v1 = fbl_sim.download()

    plotter.plot(eta1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]],\
                 u1[validDomain[2]:-validDomain[0], validDomain[3]-1:-validDomain[1]+1], \
                 v1[validDomain[2]-1:-validDomain[0]+1, validDomain[3]:-validDomain[1]]);

    fig.suptitle("FBL Time = " + "{:04.0f}".format(t) + " s", fontsize=18)

    if (i%20 == 0):
        print "{:03.0f}".format(100.0*i / T) + " % => t=" + str(t)
        fig.savefig(imgdir + "/{:010.0f}_fbl.png".format(t))
            
anim = animation.FuncAnimation(fig, animate, range(T), interval=100)
plt.close(anim._fig)
anim

In [ ]:
if make_netCDF:
    fbl_sim.cleanUp()

In [ ]:
# Kurganov-Petrova 2007
if 'kp07_sim' in globals():
    kp07_sim.cleanUp()

ghosts = np.array([2,2,2,2]) # north, east, south, west
validDomain = np.array([2,2,2,2])
if (bcSettings == 1):
    boundaryConditions = Common.BoundaryConditions()
elif (bcSettings == 2):
    boundaryConditions = Common.BoundaryConditions(2,2,2,2)
elif bcSettings == 3:
    validDomain = [10, 10, 10, 10]
    boundaryConditions = Common.BoundaryConditions(3,3,3,3, spongeCells=validDomain)
    ghosts = [10, 10, 10, 10]
        
dataShape = (ny + ghosts[0]+ghosts[2], 
             nx + ghosts[1]+ghosts[3])

eta0 = np.zeros(dataShape, dtype=np.float32, order='C');
u0 =   np.zeros(dataShape, dtype=np.float32, order='C');
v0 =   np.zeros(dataShape, dtype=np.float32, order='C');

# Bathymetry:
Hi = np.zeros((dataShape[0]+1, dataShape[1]+1), dtype=np.float32, order='C')
linearBathymetryY(Hi, dx, dy, nx+1, ny+1, ghosts, alpha, waterHeight, y0_reference_cell+0.5 )

initializeBalancedBumpOverPoint(eta0, u0, v0, # allocated buffers to be filled with data (output)
                                nx, ny, dx, dy, ghosts, # grid data
                                bump_posx, bump_posy, # relative placement of bump center
                                bump_height, bump_width_factor, # bump information
                                f, waterHeight, # parameters defined at the bump centre (coriolis force, water depth)
                                g)

#Initialize simulator
reload(KP07)
kp07_sim = KP07.KP07(cl_ctx, \
                     eta0, Hi, u0, v0, \
                     nx, ny, dx, dy, dt, \
                     g, f, r, \
                     coriolis_beta=beta, \
                     wind_stress=wind, \
                     write_netcdf=make_netCDF, \
                     boundary_conditions=boundaryConditions)

fig = plt.figure()
plotter = PlotHelper.PlotHelper(fig, x_coords, y_coords, radius, 
                                h0[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]], 
                                u0[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]], 
                                v0[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]])

def animate(i):
    if (i>0):
        t = kp07_sim.step(sub_t)
    else:
        t = 0.0
    eta1, u1, v1 = kp07_sim.download()
    
    # To increase the amplitude of the plot, increase this brighten parameter
    brighten = 1
    
    plotter.plot(brighten*(eta1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]]), 
                 brighten*u1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]], 
                 brighten*v1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]]);
    fig.suptitle("KP07 Time = " + "{:04.0f}".format(t) + " s", fontsize=18)

    if (i%20 == 0):
        print "{:03.0f}".format(100*i / T) + " % => t=" + str(t) + "\tMax h: " + str(np.max(eta1)) + \
        "\tMax u: " + str(np.max(u1)) + \
        "\tMax v: " + str(np.max(v1))
        fig.savefig(imgdir + "/{:010.0f}_kp07.png".format(t))
             
anim = animation.FuncAnimation(fig, animate, range(T), interval=100)
plt.close(anim._fig)
anim

In [ ]:
if make_netCDF:
    kp07_sim.cleanUp()